In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

### Normal Equation

In [22]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

### Gradient Descent

In [63]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((scaled_housing_data_plus_bias.shape[1], 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  5.7089496
Epoch 100 MSE =  5.0724845
Epoch 200 MSE =  5.000963
Epoch 300 MSE =  4.9501376
Epoch 400 MSE =  4.912774
Epoch 500 MSE =  4.8852344
Epoch 600 MSE =  4.8648696
Epoch 700 MSE =  4.8497643
Epoch 800 MSE =  4.838525
Epoch 900 MSE =  4.8301373


### Autodiff

In [64]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((scaled_housing_data_plus_bias.shape[1], 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  6.6078525
Epoch 100 MSE =  5.0543175
Epoch 200 MSE =  4.9808254
Epoch 300 MSE =  4.9364233
Epoch 400 MSE =  4.903802
Epoch 500 MSE =  4.8795495
Epoch 600 MSE =  4.861417
Epoch 700 MSE =  4.847815
Epoch 800 MSE =  4.837576
Epoch 900 MSE =  4.8298216


### Optimizer

In [119]:
from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n+1, 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE =  7.3232045
Epoch 100 MSE =  5.0218625
Epoch 200 MSE =  4.9438567
Epoch 300 MSE =  4.909403
Epoch 400 MSE =  4.884838
Epoch 500 MSE =  4.866351
Epoch 600 MSE =  4.852311
Epoch 700 MSE =  4.8415933
Epoch 800 MSE =  4.833375
Epoch 900 MSE =  4.8270288


In [111]:
best_theta

array([[ 0.36347306],
       [ 0.883878  ],
       [ 0.17219514],
       [-0.28710952],
       [ 0.28795215],
       [ 0.01423508],
       [-0.0450042 ],
       [-0.45935273],
       [-0.43240803]], dtype=float32)

### Feeding data in batch

#### Placeholder

In [4]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[1, 2, 3], [4, 5, 6]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 6.  7.  8.]
 [ 9. 10. 11.]]


In [5]:
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
X = tf.placeholder(dtype=tf.float32, shape=(None, n + 1), name='X')
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

housing = fetch_california_housing()
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X_data = scaled_housing_data_plus_bias.astype(np.float32)
y_data = housing.target.reshape(-1, 1).astype(np.float32)

def fetch_batch(batch_index, batch_size):
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = X_data[indices] # not shown
    y_batch = y_data[indices] # not shown
    return X_batch, y_batch

theta = tf.Variable(tf.random_uniform((n+1, 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if epoch % 100 == 0 and batch_index % 100 == 0:
                print("Epoch", epoch, "batch_index", batch_index, 
                      "MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))

    
    best_theta = theta.eval()
    final_mse = mse.eval(feed_dict={X: X_batch, y: y_batch})

Epoch 0 batch_index 0 MSE =  6.8661585
Epoch 0 batch_index 100 MSE =  4.8618217
Epoch 0 batch_index 200 MSE =  4.380103
Epoch 100 batch_index 0 MSE =  4.433922
Epoch 100 batch_index 100 MSE =  4.8083544
Epoch 100 batch_index 200 MSE =  5.4546094
Epoch 200 batch_index 0 MSE =  4.603949
Epoch 200 batch_index 100 MSE =  4.8993406
Epoch 200 batch_index 200 MSE =  4.195206
Epoch 300 batch_index 0 MSE =  4.7878222
Epoch 300 batch_index 100 MSE =  4.638152
Epoch 300 batch_index 200 MSE =  4.3889546
Epoch 400 batch_index 0 MSE =  4.7806244
Epoch 400 batch_index 100 MSE =  23.505112
Epoch 400 batch_index 200 MSE =  4.858443
Epoch 500 batch_index 0 MSE =  5.743591
Epoch 500 batch_index 100 MSE =  4.188973
Epoch 500 batch_index 200 MSE =  4.999134
Epoch 600 batch_index 0 MSE =  5.1130195
Epoch 600 batch_index 100 MSE =  5.561493
Epoch 600 batch_index 200 MSE =  5.1629915
Epoch 700 batch_index 0 MSE =  4.9809194
Epoch 700 batch_index 100 MSE =  5.02981
Epoch 700 batch_index 200 MSE =  4.6528826
Ep

In [32]:
best_theta

array([[ 0.60927844],
       [ 0.8317298 ],
       [ 0.13330474],
       [-0.22988458],
       [ 0.36170158],
       [-0.02722611],
       [-0.00466394],
       [-0.93803585],
       [-0.8783747 ]], dtype=float32)